Цель модуля - парсить следующую инфу, которой нет в обычной выгрузке: 
* цену контракта после снижения
* победителя
* 


In [2]:
import requests
from bs4 import BeautifulSoup as BS
import bs4
import time
from fake_useragent import UserAgent
import re 
#from decimal import Decimal
import numpy as np
import pandas as pd
import shutil
import os

In [3]:
link_main_pattern = 'http://zakupki.gov.ru'
# link_pattern_fz44_order_info = 'http://zakupki.gov.ru/epz/order/notice/ea44/view/common-info.html?regNumber='
link_pattern_fz44_result_info = 'http://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber='
# link_pattern_fz44_documents = 'http://zakupki.gov.ru/epz/order/notice/ea44/view/documents.html?regNumber='
# link_pattern_fz44_contract_info = 'http://zakupki.gov.ru/epz/contract/extendedsearch/results.html?searchString=&orderNumber='
link_pattern_fz44_contruct = 'http://zakupki.gov.ru/epz/contract/contractCard/document-info.html?reestrNumber='
link_pattern_fz223 = 'http://zakupki.gov.ru/223/purchase/public/purchase/info/common-info.html?regNumber='

### NEW 

link_fz223_documents = 'http://zakupki.gov.ru/223/purchase/public/purchase/info/documents.html?regNumber='

### Перечень локальных функций:

In [4]:
def get_page(link):
    page = requests.get(link, headers={'User-Agent': UserAgent().chrome})
    page.encoding = 'utf8'
    page = page.text
    soup = BS(page)
    while soup.title.text == ' Страница не найдена ':
        time.sleep(5)
        get_page(link)
    return soup

In [5]:
def clear_text(content):
    content = re.sub(r'[\n]+', '', content) # Очистили от переносов
    content = re.sub(r'[\s]{2,}', '', content) # Очистили от лишних пробелов
    content = re.sub(r'\xa0', '', content) # Соединили цену
    content = re.sub(r',', '.', content) # изменили запись для возможности преобрадования str в decimal
    content = re.sub(r' Российский рубль', '', content) # убираем валюту
    return content

In [78]:
def get_win_info_44(order_num):
    link = link_pattern_fz44_result_info + order_num
    soup = get_page(link)
    winner = soup.find_all('td')[-3].text # опытным путём определил, где находится наименование победителя
    winner = clear_text(winner)
    
    winner_price = soup.find_all('td')[-2].text
    winner_price = clear_text(winner_price)
    if winner_price != '':
        winner_price = float(winner_price)

        contruct_link_div = soup.find_all('a', href=True)[-1]
        link_to_contruct_info = link_main_pattern + contruct_link_div.get('href') # парсим ссылку на страницу
        num_of_contruct = contruct_link_div.text.replace('№','') #  парсим номер контракта и освобождаем его от значка №


        df = pd.DataFrame({
        'Реестровый номер закупки': [order_num],
        'победитель': [winner],
        'цена контракта после снижения': [winner_price],
        'номер контракта': [num_of_contruct]
        })
    
        return df
    

In [7]:
def get_empty_dataframe():
    df = pd.DataFrame({
    'Реестровый номер закупки': [],
    'победитель': [],
    'цена контракта после снижения': [],
    'номер контракта': []
    })
    
    return df

In [8]:
def get_contruct44(contruct_num):
    link = link_pattern_fz44_contruct + contruct_num
    soup = get_page(link)

    contruct_div = soup.find_all('a', class_='')[2] 
    link_to_contruct = contruct_div.get('href')
    file_name = contruct_div.get('title')

    file_extension = re.findall(r'\.[a-z]+', file_name)[0]
    directory = 'contructs/' 
    filename =  directory + contruct_num + file_extension
    file = requests.get(link_to_contruct, stream=True, headers={'User-Agent': UserAgent().chrome})
    
    if file.status_code == 200:
        with open(filename, 'wb') as f:
            file.raw.decode_content = True
            shutil.copyfileobj(file.raw, f)

In [81]:
def get_contruct233(contruct_num):
    link = link_fz223_documents + order_num
    soup = get_page(link)
    documents_div = soup.find_all('a', class_='epz_aware')
    directory = 'C:/Users/v.mazeiko/Documents/Парсинг Госзаупок/contructs/' + order_num

    if not os.path.isdir(directory):
        os.mkdir(directory) 

    for i in range(0, len(documents_div)-1):
        link_to_file = link_main_pattern + documents_div[i].get('href') 
    
        file = requests.get(link, stream=True, headers={'User-Agent': UserAgent().chrome})
        file.encoding = 'utf-8'
    
        if file.headers.get('Content-Disposition') is not None:
            file_extension = re.findall(r'\.[a-z][^\n]+', file.headers.get('Content-Disposition'))
        else:
            file_extension = ''
        file_name = clear_text(documents_div[i].text) + file_extension
        save_here = directory + '//' + file_name
        if file.status_code == 200:
            with open(save_here, 'wb') as f:
                file.raw.decode_content = True
                shutil.copyfileobj(file.raw, f)    

### Тренировочные данные:

In [99]:
test_data = [
 '31807326700',
 '31807270424',
 '31807200018',
 '31807199705',
 '31807151989',
 '31807219596',
 '31807153786',
 '31807147180',
 '31807002991']

### * * * Сборочный стол * * *

In [63]:
order_num = str(test_data[0])

In [106]:
def get_win_info_233(order_num, pause):
    link_fz223_contructs_links = 'http://zakupki.gov.ru/223/purchase/public/purchase/info/contractInfo.html?regNumber='
    link = link_fz223_contructs_links + str(order_num)
    
    soup = get_page(link)
    
    link = soup.find_all('li')[0].get('onclick')
    if link is None:
        df = get_empty_dataframe()
        df['Реестровый номер закупки'] = order_num
        return df
    
    link = re.sub(r'window\.open\(\'|\'\)', '', link)
    link = re.sub(r'view-common', r'view-subject', link)
    link = link_main_pattern + link
    soup = get_page(link)
    time.sleep(pause)

    winner = soup.find_all('td')[-7].text
    winner_price = soup.find_all('td')[10].text

    if winner_price != '':
            #winner_price = float(winner_price)
            winner_price =  winner_price
    
            contruct_link_div = soup.find_all('a', href=True)[-1]
            link_to_contruct_info = link_main_pattern + contruct_link_div.get('href') # парсим ссылку на страницу
            num_of_contruct = re.findall(r'[\d-]+', clear_text(soup.h1.string))[0] #  парсим номер контракта
    
    
            df = pd.DataFrame({
            'Реестровый номер закупки': [order_num],
            'победитель': [winner],
            'цена контракта после снижения': [winner_price],
            'номер контракта': [num_of_contruct]
            })
            
            return df

### * * * THE END * * *

### Проверка

In [100]:
df = get_empty_dataframe()
df

,Реестровый номер закупки,победитель,цена контракта после снижения,номер контракта


In [107]:
for order in test_data:
    tmp_df = get_win_info_233(order, 2)
    print
    df= pd.concat([df, tmp_df])
    time.sleep(2)
    

In [108]:
df

,Реестровый номер закупки,победитель,цена контракта после снижения,номер контракта
0,31807326700,ООО «ЭП «Меркурий»,13008,31807326700-01
0,31807270424,"Общество с ограниченной ответственностью ""Крон...",2.32331e+06,31807270424-01
0,31807151989,"Общество с ограниченной ответственностью ""ЭКО ...",284850,31807151989-01
0,31807326700,ООО «ЭП «Меркурий»,13008,31807326700-01
0,31807270424,"Общество с ограниченной ответственностью ""Крон...",2323309.56,31807270424-01
0,31807151989,"Общество с ограниченной ответственностью ""ЭКО ...",284850,31807151989-01
0,31807219596,"ООО ""ОРКО-инвест""","цена размещения 1 м3 отходов III, IV классов о...",31807219596-01
0,31807153786,"Общество с ограниченной ответственностью ""СТО""",149000,31807153786-01
0,31807147180,"Общество с ограниченной ответственностью ""Сорэ...",403351.5,31807147180-01
0,31807002991,"Общество с ограниченной ответственностью ""СОРЭ...",391500,31807002991-01


In [103]:
df['Реестровый номер закупки'] ==

0    31807326700
0    31807270424
0    31807151989
Name: Реестровый номер закупки, dtype: object

# ПОМОЙКА

In [ ]:
tag = soup.td
tag

In [ ]:
tag.attrs

In [ ]:
tag.contents

In [ ]:
for child in tag.descendants:
    print(child)

In [ ]:
len(list(soup.children))

In [ ]:
len(list(soup.descendants))

In [ ]:
soup.find_all('div', class_='contentTabBoxBlock') # нашёл бокс с интересующей инфой

In [ ]:
b = soup.find_all('td')
for i in range(len(b)):
    print(f'**************{i}****************')
    print(b[i])

In [ ]:
b[16].text # winner

In [ ]:
soup.find_all('td')[16].text

# СПРАВОЧНИК

In [ ]:
# soup.find_all('a')  - все ссылки

### Интересная штучка из  http://qaru.site/questions/15235681/beautifulsoup4-documentation-example-doesnt-work/24168590  для поиска определённых тагов

In [ ]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')
a.find_all(has_class_but_no_id)

In [ ]:
# образец кода для парсинга слов в вики (первого порядка)
def get_links(topic):
    link = 'https://ru.m.wikipedia.org/wiki/' + topic.capitalize()
    spam = get(link).text
    soup = BS(spam, 'html.parser')
    link_example = """ <a href="LINK">CONTENT</a> """
    agg = soup.find_all("a")
    links = []
    for n in agg:
        lnk = n.get("href")
        if re.fullmatch(r'\/wiki\/(?:)[^\',\n]*|\/w\/(?:)[^\',\n]*', lnk) is not None:
            links.append('https://ru.m.wikipedia.org' + lnk)
        else:
            continue
    links.append(link)

    return links

In [ ]:
# цикл, возвращающий ссылки:
for i in soup.find_all('a', href=True):
    print(i['href'])